# Demo of Classless ProBP

Based on Alexander Papen & Bazyli Szymański [2020] "Classless Continuous RM: Exploration of
Methods and Assumptions"


In [158]:
# Basic structures
# These are siilar to the core classes in PassengerSim
#
from math import exp, log, sqrt
from dataclasses import dataclass, field
import pandas as pd

@dataclass
class Item:
    item_no: int
    path_index: int
    tf: int
    mkt: str
    contrib: float
    fcst_mean: float
    fcst_std_dev: float
    joint_mean: float = 0.0
    joint_std_dev: float = 0.0
    joint_fare: float = 0.0
    prot: float = 0.0
    raw_bp: float = 0.0
    

@dataclass
class Leg:
    flt_no: int
    orig: str
    dest: str
    bid_price: float = 1.0
    capacity: int = 10
    items: list[Item] = field(default_factory=list)
    
    def str_items(self):
        return pd.DataFrame(self.items.round(2))


@dataclass
class Path:
    path_no: int
    orig: str
    dest: str
    leg_index: list[int]
    q_demand: list[float]
    q_fare: float
    y_fare: float
    sum_bp: float = 0.0
    q_std_dev: list[float] = None
    f_star: list[float] = None
    d_star: list[float] = None

In [8]:
# Setup the example from the presentation

def setup():
    num_tf = 4
    frat5 = [1.5, 2.0, 2.5, 3.0]
    k_factor = 0.5
    
    legs = [
        Leg(1, "AAA", "BBB"),
        Leg(2, "BBB", "CCC")
    ]
    
    paths = [
        Path(1, "AAA", "BBB", [0], [5, 5, 5, 5], 100.0, 800.0),
        Path(2, "BBB", "CCC", [1], [7, 7, 7, 7], 75.0, 600.0),
        Path(3, "AAA", "CCC", [0, 1], [5, 5, 5, 5], 125.0, 1000.0)
    ]

    for p in paths:
        p.f_star = [0.0] * num_tf
        p.d_star = [0.0] * num_tf
        p.q_std_dev = [round(d * k_factor, 2) for d in p.q_demand]

    return legs, paths, num_tf, frat5
    

# Functions for Classless ProBP



In [153]:
# STEP 1 
#  - Calculate the optimal fare (f_star) based on bid price and elasticity
#  - Estimate the demand (d_star) that we'll get for that fare level
#
def calculate_fare(debug=False):
    for p in paths:
        if debug:
            print("calculate_fare")
        for t in range(num_tf):
            p.sum_bp = 0.0
            for l in p.leg_index:
                p.sum_bp += legs[l].bid_price
            tmp = p.q_fare * (frat5[t] - 1.0) / log(2.0)
            p.f_star[t] = min(p.y_fare, max(p.q_fare, tmp) + p.sum_bp)
            p_wtp = min(1.0, exp((log(0.5) * (p.f_star[t] - p.q_fare)) / ((frat5[t] - 1.0) * p.q_fare)))
            p.d_star[t] = p.q_demand[t] * p_wtp
            if debug:
                print(f"    tf={t}, sum_bp={round(p.sum_bp, 2)}, f_star={round(p.f_star[t], 2)}, d_star={round(p.d_star[t], 2)}")

In [154]:
# Step 2 - Proration
#
def proration(debug=False):
    if debug:
        print(f"Proration")

    for leg in legs:
        leg.items = []
    
    for p in paths:
        mkt = f"{p.orig}-{p.dest}"
        for l in p.leg_index:
            leg = legs[l]
            for t in range(num_tf):
                contrib = p.f_star[t] * leg.bid_price / p.sum_bp
                mean = p.d_star[t]
                var = p.q_std_dev[t] ** 2
                var *= mean / p.q_demand[t]
                std_dev = sqrt(var)
                item = Item(0, p.path_no, t, mkt, contrib, mean, std_dev)
                leg.items.append(item)
    
    # Sort the items by contribution
    for leg in legs:
        leg.items.sort(reverse=True, key=lambda x: x.contrib)
        if debug:
            print(f"Leg: {leg.orig}-{leg.dest}")
            print("    ", leg.str_items())


In [172]:
# Step 3 - EMSRc calculations
#  - We estimate Fare * P(demand > capacity)
#
from statistics import NormalDist
nd = NormalDist(mu=0.0, sigma=1.)
alpha = 0.3

def emsrc(debug=False):
    max_diff = 0.0
    for leg in legs:
        if debug:
            print(f"EMSRc, Leg = {leg.orig}-{leg.dest}")

        bid_price = 0.0
        agg_mean, agg_variance, agg_fare, prior_contrib, prior_prot = 0, 0, 0, 0, 0
        for item in leg.items:
            agg_mean += item.fcst_mean
            agg_variance += item.fcst_std_dev ** 2
            agg_fare += item.contrib * item.fcst_mean

            item.joint_mean = agg_mean
            item.joint_std_dev = sqrt(agg_variance)
            item.joint_fare = agg_fare / agg_mean

        for i in range(len(leg.items) - 1):
            item = leg.items[i]
            if prior_prot < leg.capacity:
                fare_ratio = leg.items[i+1].contrib / item.joint_fare
                fare_ratio = min(0.9999999, max(0.0000001, fare_ratio))
                item.prot = NormalDist(mu=item.joint_mean, sigma=item.joint_std_dev).inv_cdf(1.0 - fare_ratio)
                prior_prot = item.prot
                item.raw_bp = item.joint_fare * (1 - nd.cdf((leg.capacity - item.joint_mean) / item.joint_std_dev))
                item.raw_bp = min(item.raw_bp, prior_contrib)
                bid_price = max(bid_price, item.raw_bp)
                prior_contrib = item.contrib
        old_bp = leg.bid_price
        leg.bid_price = alpha * bid_price + (1.0 - alpha) * leg.bid_price
        max_diff = max(abs(old_bp - leg.bid_price), max_diff)
        if debug:
            print(pd.DataFrame(leg.items)[["mkt", "contrib", "fcst_mean", "fcst_std_dev", "prot", "raw_bp"]].round(2))
    return max_diff
    

In [174]:
# ProBP
#  - iterate until convergence
#

max_iter = 50
convergence_limit = 5

legs, paths, num_tf, frat5 = setup()
print(pd.DataFrame(paths).round(2))
for n in range(max_iter):
    print(f"********** Iteration: {n} **********")
    calculate_fare(debug=False)
    # print(pd.DataFrame(paths)[["orig", "dest", "sum_bp", "f_star"]].round(2))
    proration(debug=False)
    max_diff = emsrc(debug=False)
    print(pd.DataFrame(legs)[["flt_no", "orig", "dest", "bid_price"]].round(2))
    print(f"max_diff={round(max_diff, 2)}")
    if max_diff < convergence_limit:
        print("***** Converged *****")
        break
    print("")

info = []
for p in paths:
    for tf, f in enumerate(p.f_star):
        info.append({"orig": p.orig,
                     "dest": p.dest,
                     "tf": tf,
                     "f_star": f})
print("***** Paths and fares")
print(pd.DataFrame(info).round(2))

   path_no orig dest leg_index      q_demand  q_fare  y_fare  sum_bp  \
0        1  AAA  BBB       [0]  [5, 5, 5, 5]   100.0   800.0     0.0   
1        2  BBB  CCC       [1]  [7, 7, 7, 7]    75.0   600.0     0.0   
2        3  AAA  CCC    [0, 1]  [5, 5, 5, 5]   125.0  1000.0     0.0   

              q_std_dev                f_star                d_star  
0  [2.5, 2.5, 2.5, 2.5]  [0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0]  
1  [3.5, 3.5, 3.5, 3.5]  [0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0]  
2  [2.5, 2.5, 2.5, 2.5]  [0.0, 0.0, 0.0, 0.0]  [0.0, 0.0, 0.0, 0.0]  
********** Iteration: 0 **********
   flt_no orig dest  bid_price
0       1  AAA  BBB      41.80
1       2  BBB  CCC      40.74
max_diff=40.8

********** Iteration: 1 **********
   flt_no orig dest  bid_price
0       1  AAA  BBB      76.07
1       2  BBB  CCC      76.18
max_diff=35.44

********** Iteration: 2 **********
   flt_no orig dest  bid_price
0       1  AAA  BBB     106.07
1       2  BBB  CCC     102.85
max_diff=30.0

*